In [1]:
%run functions.ipynb
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train, X_test, y_train, y_test = load_train_and_test_parts()
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print(
    "(1, 0) labels count in train test: {} {}".format(
        (np.count_nonzero(y_train == 1), np.count_nonzero(y_train == 0)),
        (np.count_nonzero(y_test == 1), np.count_nonzero(y_test == 0)),
    )
)

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [3]:
scoring = 'roc_auc'
cv = 4
cv_out = 5
cv_in = 2
n_iter = 1
n_jobs = 3

In [4]:
rlr_params = {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
rlr_cv = RandomizedSearchCV(RandomLogisticsRegressions(penalty='l1'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=rlr_params, random_state=random_state)

clf_rlr = fit_clf_print_scores(rlr_cv, X_train, y_train, X_test, y_test)

{'n_variables': 500, 'n_estimators': 250, 'C': 0.7000000000000001}


,ROC AUC,Precision,Recall,F1
train,0.974,0.961,0.855,0.905
test,0.825,0.739,0.654,0.694


In [5]:
all_scores_names = ['roc_auc', 'precision', 'recall', 'f1']
estimated_scores = {}
randomized_cvs = {}
models = {
    'Random logistic regression': {
        'model': RandomLogisticsRegressions(penalty='l1'),
        'params': {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
    }
}

In [6]:
for model_name, model_specification in models.items():
    rcv = RandomizedSearchCV(estimator=model_specification['model'], param_distributions=model_specification['params'],
                             cv=cv_in, scoring=scoring, n_iter=n_iter, iid=False, random_state=random_state)
    randomized_cvs[model_name] = rcv

In [7]:
for name, rcv in randomized_cvs.items():
    nested_scores = cross_validate(rcv, X_train, y_train, scoring = all_scores_names, cv=cv_out)
    estimated_scores[name] = {}
    for score_name in all_scores_names:
        estimated_scores[name][score_name] = {
            'mean': np.mean(nested_scores['test_' + score_name]),
            'std': np.std(nested_scores['test_' + score_name])
        }
    print('Done for ' + name)

Done for Random logistic regression


In [8]:
print_summarized_scores(estimated_scores, models, latex=True)

,ROC AUC,Precision,Recall,F1
Random logistic regression,0.809 $\pm$ 0.028,0.709 $\pm$ 0.043,0.633 $\pm$ 0.019,0.668 $\pm$ 0.023


\begin{tabular}{lllll}
\toprule
{} &            ROC AUC &          Precision &             Recall &                 F1 \\
\midrule
Random logistic regression &  0.809 \$\textbackslash pm\$ 0.028 &  0.709 \$\textbackslash pm\$ 0.043 &  0.633 \$\textbackslash pm\$ 0.019 &  0.668 \$\textbackslash pm\$ 0.023 \\
\bottomrule
\end{tabular}

